# Excercise 1: Basic Python API #

This exercise has been created by [M. Siggel](https://github.com/rainman110/tigl-workshop/commits?author=rainman110) and is part of an exercise collection on [GitHub](https://github.com/rainman110/tigl-workshop).

The goal of this excercise is to get familiar with the TiGL API and its concepts.
 - You should traverse trough the CPACS tree, get the wing shape. 
 - Create a box with OpenCASCADE
 - Export both with the TiGL IGES exporter with custom layers
 
## Loading the modules ##
As usual, load the required TiGL and TiXI modules

In [ ]:
from tixi3 import tixi3wrapper
from tigl3 import tigl3wrapper
import tigl3.configuration

Now open a the CPACS file simpletest.xml using the "old" high-level python API.

In [ ]:
tixi_h = tixi3wrapper.Tixi3()
tigl_h = tigl3wrapper.Tigl3()

tixi_h.open("./resources/simpletest.cpacs.xml")
tigl_h.open(tixi_h, "")

## CPACS Traversal ##
First, get the configuration manager from tigl3.configuration and retrieve the aircraft configuration.

__Hint: CCPACSConfigurationManager_get_instance() gets the global configuration manager object__

In [ ]:
# get the configuration manager
mgr = tigl3.configuration.CCPACSConfigurationManager_get_instance()

Get the aircraft config. Now, both "old" and "new" API come together, when we have to pass the TiGL handle to the configuration manager.

In [ ]:
aircraft_config = mgr.get_configuration(tigl_h._handle.value)

Get the wing from the aircraft confiuration and print its wing uid. Check, what method the wing object offers.

In [ ]:
wing = aircraft_config.get_wing(1)

In [ ]:
print(wing.get_uid())

In [ ]:
wing.get_aspect_ratio()

In [ ]:
help(wing)

## Wing trailing edge renaming ##

Now next tasks are:
 - Get the wing shape
 - Print all face names of the wing
 - Rename the trailing edge faces

In [ ]:
wing_shape = wing.get_loft()

In [ ]:
for iface in range(0, wing_shape.get_face_count()):
    if wing_shape.get_face_traits(iface).name() == "TrailingEdge":
        wing_shape.get_face_traits(iface).set_name("EdgeTrail")
    print(wing_shape.get_face_traits(iface).name())

## OCC plotting ##

We use openCascade to plot the wing shape.

In [ ]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [ ]:
renderer = JupyterRenderer(size=(700,700))
renderer.DisplayShape(wing_shape.shape(), transparency=True)
renderer.Display()

## Create named shape with a box ##

- Create a box with OpenCASCADE. You can use the BRepPrimAPI_MakeBox class from the OCC.BRepPrimAPI package.
- Then, create a named shape from the TopoDS_Shape.

In [ ]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
box = BRepPrimAPI_MakeBox(0.3, 0.8, 0.3).Solid()

In [ ]:
from tigl3.geometry import CNamedShape
box_ns = CNamedShape(box, "MyBox")

## Export of the shapes ##
Finally, you have to export both shapes into the IGES format. Both shapes should have a different IGES layer!

You can inspect the result in TiGL Viewer by drag-and-dropping the igs file into the viewer.

In [ ]:
from tigl3.exports import create_exporter, IgesShapeOptions
iges_exporter = create_exporter("igs")
iges_exporter.add_shape(wing_shape, IgesShapeOptions(1111))
iges_exporter.add_shape(box_ns, IgesShapeOptions(2222))
iges_exporter.write("ex1.igs")